In [1]:
import os
import sys

import pandas as pd

sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")

import datetime
from utils import get_path, DATETIME_FORMAT

In [4]:
from universal_connection import UniversalConnection, DBType

In [8]:
import re

In [61]:
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
# set(stopwords.words('russian'))

In [2]:
sql = """
with temp as (
select	a.id app_id,
		a.report_comment,
		case when pc.product_id in (
		            		select id from product
		            		where full_name like '%ВЭБ%') then 'VEBMED'
		        else 'ПРОЧИЕ'
		end promo_type
from appointment a 
left join promotion p
	      on a.promotion_id = p.id
	    left join product_condition pc
	      on pc.id = p.product_condition_id
	      --
	where a.good
      and not (
      	a.patient_id in (
	      	select patient_id
	      	from patient_categories pcat
	      	inner join reference rf
	      	   on rf.id=pcat.reference_id
	      	where rf.code='TEST'
	      	)
	    or lower(a.report_comment)='тест'
      	)
)
select *
from temp 
where promo_type = 'VEBMED'





"""

In [5]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
df_sql = connection.query(sql)
connection.close()

2020-05-28 17:54:38 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-05-28 17:54:38 creating ssh tunnel to 172.16.100.19 as root...
2020-05-28 17:54:38 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 33587
2020-05-28 17:54:38 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-05-28 17:54:38 @telemed query:
                    with temp as (
				    select	a.id app_id,
				    		a.report_comment,
				    		case when pc.product_id in (
				    		            		select id from product
				    		            		where full_name like '%ВЭБ%') then 'VEBMED'
				    		        else 'ПРОЧИЕ'
				    		end promo_type
				    from appointment a 
				    left join promotion p
				    	      on a.promotion_id = p.id
				    	    left join product_condition pc
				    	      on pc.id = p.product_condition_id
				    	 

In [6]:
df_sql.head()

,app_id,report_comment,promo_type
0,127919,О.ринит? у ребенка 3-х мес жизни\nРекомендован...,VEBMED
1,144566,"консультация уролога - очная,\r\nузи простаты,...",VEBMED
2,135952,Диагностическая гипотеза : НМЦ по типу олигоме...,VEBMED
3,173510,Жалобы: на головную боль в затылочной области....,VEBMED
4,152628,на основании жалоб. данных анамнеза и показате...,VEBMED


In [62]:
d = dict()

In [63]:
stop_words = set(stopwords.words('russian'))

In [64]:
for i in range(df_sql.shape[0]):
    temp = df_sql['report_comment'][i]
    temp = re.sub(r'[^\w\s]+|[\d]+', r'',temp.lower()).replace('ё', 'е').strip().split()
    temp = [x for x in temp if not x in stop_words]
    temp = [x for x in temp if len(x) > 2]
    
    for word in temp:
        d[word] = d.get(word, 0) + 1

In [65]:
words = pd.DataFrame.from_dict(d, orient='index').sort_index().reset_index().rename(columns={'index':'word', 0:'amount'})

In [68]:
words = words.sort_values(by=['amount'], ascending=False).reset_index(drop=True)

In [69]:
words

,word,amount
0,консультация,31101
1,рекомендовано,30014
2,диагностическая,29459
3,гипотеза,28084
4,жалобы,23214
...,...,...
107631,теба,1
107632,театральным,1
107633,театральную,1
107634,разводим,1


In [71]:
words.to_excel('words.xlsx', index=False)